# 1.0 Data Cleaning

## 1.1 Importing Stuff

Import date of raw data file `CA.US-Social.csv`:  2024-11-17

Source:  https://webapps.cihr-irsc.gc.ca/decisions/p/main.html?lang=en#fq={!tag=theme2}theme2%3A%22Social%20%2F%20Cultural%20%2F%20Environmental%20%2F%20Population%20Health%22&fq={!tag=country}country%3ACanada%20%20%20OR%20%20%20country%3A%22United%20States%20of%20America%22&sort=namesort%20asc&start=0&rows=20

In [ ]:
# installations
## section 1.1
import pandas as pd

## section 1.2


In [33]:
data = pd.read_csv("raw data/CA.US-Social.csv", encoding='latin1')
grant_count = data.shape[0] # will use this in section 1.2

In [27]:
# clean numeric columns
numeric_cols = ['CIHR_Contribution', 'CIHR_Equipment']
for column in numeric_cols:
    data[column] = data[column].replace({'\$': '', ',': ''}, regex=True)

# remove 0's
data['CIHR_Contribution'] = pd.to_numeric(data['CIHR_Contribution'], errors='coerce') 
data = data.query("CIHR_Contribution != 0")

## 1.2 Scraping Data

So the search results from which I imported the data contains links to the information on the papers.  Within these links, the abstracts can be found, so once we can scrape that we're golden.

Issues:
- search results are split into 352 pages, with 20 results per page
- the links to the papers are independent of their positions on the search results

Link structure of search results:

`https://webapps.cihr-irsc.gc.ca/decisions/p/main.html?lang=en#fq={!tag=theme2}theme2%3A%22Social%20%2F%20Cultural%20%2F%20Environmental%20%2F%20Population%20Health%22&fq={!tag=country}country%3ACanada%20%20%20OR%20%20%20country%3A%22United%20States%20of%20America%22&sort=namesort%20asc&start=`**0**`&rows=20` where `row` is a multiple of 20.  The maximum n should be the  max possible integer that satisfies 20n < [count of grants] 



Link structure of papers:

`https://webapps.cihr-irsc.gc.ca/decisions/p/project_details.html?applId=`**462354**`&lang=en` where the `applID` is unique to each paper.

Proposed steps:
1. webscrape the `applID` of all papers
2. use the `applID` to obtain the soup for each grant
3. isolate abstract, project name, researcher(s) name, contribution count

Actual steps:
1. download chromedriver here https://storage.googleapis.com/chrome-for-testing-public/131.0.6778.69/win64/chromedriver-win64.zip and extract.  The extract location will differ based on operating system (Windows or Mac)
2. initialize driver for selenium.  This step is required since the site is dynamically loaded
3. load and scrape the `applID`s of all n pages of the search result
    
    a. save to `data/applids.csv`
4. use the `applID`s to scrape the abstract, project name, researcher(s) names, contribution count

### 1.2.1 Scraping applIDs

This shit took 27 mins to run.  Don't run it again unless necessary LUL (turn the raw blocks back into Python blocks if you need to run again)

### 1.2.2 Scraping everything else

In [92]:
grants_info = pd.read_csv("data/applids.csv")
grants_applIDs = grants_info['applId'].to_list()